In [1]:
pip install unidecode

In [2]:
import pandas as pd
import glob
import os
import re
from unidecode import unidecode

In [3]:
#LEGGO FILE EXCELS (2023, 2024)
file_excel = glob.glob(os.path.join('*.xlsx'))

lista_df = []
body_clean_regex = r'https?://[^\s]+|[@_"\\.,:;!?“”\']'

#implementare meccanismo per gestire id incrementale qualora si inseriscano più dati o social

for file in file_excel:
    df = pd.read_excel(file)
    df = df.drop_duplicates()
    year = file.split("_")[-1].replace(".xlsx","")
    #escludo tweets più recenti dei quali non so la data
    df = df[~df['date'].str.contains('h')]
    #calcolo data a partire dalla data in formato stringa
    df['date'] = df['date'].str.split(", ").apply(lambda x: x[0]) + " " + year
    df['date'] = pd.to_datetime(df["date"], format="%b %d %Y")
    #applico filtro regex, sostituisco a capo con spazio e applico funzione unicode per trasformare le lettere con l'accento nelle rispettive senza
    df['body_cleaned'] = df['body'].apply(lambda x: unidecode(re.sub(body_clean_regex, '', x).lower())).replace("\n",' ', regex = True)
    df['source'] = 'twitter'
    lista_df.append(df)

#assegno id a post
df_tweets = pd.concat(lista_df, ignore_index=True).reset_index().rename(columns={'index': 'id_post'})
df_tweets

,id_post,body,date,count_comments,count_retweets,count_likes,count_views,body_cleaned,source
0,0,La razza #Girgentana è tutelata da un Presidio...,2024-08-26,NaN,NaN,NaN,19,la razza #girgentana e tutelata da un presidio...,twitter
1,1,Siccità in Sicilia. L’allarme degli allevatori...,2024-08-26,NaN,NaN,NaN,16,siccita in sicilia l'allarme degli allevatori ...,twitter
2,2,Siccità e crisi idrica in #Sicilia #CrisiIdric...,2024-08-26,NaN,NaN,1,19,siccita e crisi idrica in #sicilia #crisiidric...,twitter
3,3,Per Giubilei \n@giubileif\n tutto quello che i...,2024-08-26,1.0,9,16,353,per giubilei giubileif tutto quello che i ci...,twitter
4,4,"Siccità: allevatori capre girgentane, viviamo ...",2024-08-26,1.0,NaN,1,70,siccita allevatori capre girgentane viviamo in...,twitter
...,...,...,...,...,...,...,...,...,...
70464,70464,Coglione la siccità c'è pure in Italia,2023-03-15,NaN,NaN,NaN,8,coglione la siccita ce pure in italia,twitter
70465,70465,"Siccità, Mattarella lancia l’allarme: “Va affr...",2023-03-15,NaN,NaN,NaN,6,siccita mattarella lancia l'allarme va affront...,twitter
70466,70466,@repubblica\n racconta la drammatica situazion...,2023-03-15,NaN,NaN,NaN,9,repubblica racconta la drammatica situazione ...,twitter
70467,70467,"""La siccità è un problema che aumenta anche i ...",2023-03-15,NaN,6,10,2.9K,la siccita e un problema che aumenta anche i f...,twitter


In [4]:
#RECUPERO HASHTAGS
#splitto testo in un vettore di parole e ciclo il vettore tenendo solo hashtags
df_hashtags = df_tweets['body_cleaned'].str.split(" ").apply(lambda x : [word for word in x if word.startswith('#')]).reset_index()
#faccio una pivot in modo da avere una riga per coppia hastagh/tweet
df_hashtags = df_hashtags.explode('body_cleaned')
df_hashtags.rename(columns = {'body_cleaned':'hashtag', 'index':'id_post'}, inplace = True)
df_hashtags

,id_post,hashtag
0,0,#girgentana
0,0,#sicilia
0,0,#siccita
0,0,#caltanissetta
0,0,#ansaterragusto
...,...,...
70465,70465,NaN
70466,70466,#siria
70466,70466,#guerra
70467,70467,NaN


In [5]:
df_hashtags.to_excel('hashtags_tableau.xlsx', index=False)
df_tweets.to_excel('tweets_tableau.xlsx', index=False)

In [ ]:
df_tweets[df_tweets['count_views']]